In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import config

from config import user
from config import admin

In [2]:
netflix_file = "Resources/Netflix_Movies.csv"
netflix_df = pd.read_csv(netflix_file)
netflix_df.head(2)

,Unnamed: 0,title,director,cast,country,release_year,rating
0,0,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2020,PG-13
1,1,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,2021,PG


In [3]:
c_netflix_df = netflix_df.drop(columns=["Unnamed: 0"])

In [4]:
c_netflix_df = c_netflix_df.rename(columns={"cast": "movie_cast"})

In [5]:
c_netflix_df['id'] = c_netflix_df.index
c_netflix_df.head()

,title,director,movie_cast,country,release_year,rating,id
0,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2020,PG-13,0
1,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,2021,PG,1
2,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",1993,TV-MA,2
3,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021,PG-13,3
4,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021,TV-MA,4


In [6]:
protocol = 'postgresql'
username = user
password = admin
host = 'localhost'
port = 5432
database_name = 'movies'

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
        
engine = create_engine(rds_connection_string)

In [7]:
inspector = inspect(engine)

inspector.get_table_names()

['netflix', 'disney_plus']

In [8]:
c_netflix_df.to_sql(name='netflix', con=engine, if_exists='append', index=False)

In [9]:
pd.read_sql_query('select * from netflix', con=engine).head()

,id,title,director,movie_cast,country,release_year,rating
0,0,Dick Johnson Is Dead,Kirsten Johnson,None,United States,2020,PG-13
1,1,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",None,2021,PG
2,2,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",1993,TV-MA
3,3,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021,PG-13
4,4,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021,TV-MA
